In [3]:
import re
import pandas as pd
#from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
short = pd.read_csv('../high-charge-density22.csv')
short['Class'] = 1
# Apply filters: proline > 15%, length > 6 and < 15
pos = short[
    (short['Sequence'].str.len() > 6) &
    (short['Sequence'].str.len() < 40)
]

pos = pos.sample(n=min(4526, len(pos)), random_state=40)
pos

,Sequence,Class
1602,FWRRYKKVKKYRRWF,1
3413,VDKPPYLPRVRPPRRIYNR,1
596,WWKRWKRIRRIFMMV,1
913,LLCKLKCKLKL,1
2492,KLKAGLAKWKAGLAKLKAGLA,1
...,...,...
575,VQLRCRVCVIRK,1
1509,WRLWRLWRLWRLWRL,1
1381,LARFVLRILKYGFK,1
3620,VAKKLAKLAKKLAKLAL,1


In [4]:
import re
import pandas as pd
#from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
short = pd.read_csv('../neg-high-charge-density22.csv')
short['Class'] = 0
neg = short[
    (short['Sequence'].str.len() > 6) &
    (short['Sequence'].str.len() < 40)
]
neg
# Display the result
neg
# Sample 3534 rows

neg

,Sequence,Class
0,EREREKRFSFFKKNK,0
1,GRKRRQTSLTDFYHSKRRLVFCKRKP,0
2,ARRRRRHASTKLKRRRRRRRHGKKSHK,0
3,ARRRRRSSRPQRRRRRRRHGRRRRGRR,0
4,PRRRTRRASRPVRRRRPRRVSRRRRARRRR,0
...,...,...
4523,KSLSTGKSKSFFVRQTNKS,0
4524,KSKSFFVRQTKKS,0
4525,KSLSTGKSKSFFVRQTKKS,0
4526,FSSEKLKARKEKKSRKQAPY,0


In [5]:
concatenated_df = pd.concat([pos,neg], ignore_index=True)
concatenated_df = concatenated_df.drop_duplicates(subset='Sequence', keep=False)
concatenated_df

,Sequence,Class
0,FWRRYKKVKKYRRWF,1
1,VDKPPYLPRVRPPRRIYNR,1
2,WWKRWKRIRRIFMMV,1
3,LLCKLKCKLKL,1
4,KLKAGLAKWKAGLAKLKAGLA,1
...,...,...
9047,KSLSTGKSKSFFVRQTNKS,0
9048,KSKSFFVRQTKKS,0
9049,KSLSTGKSKSFFVRQTKKS,0
9050,FSSEKLKARKEKKSRKQAPY,0


In [6]:
import re
# Clean comma-separated sequences (take the first part)
concatenated_df['Sequence'] = concatenated_df['Sequence'].str.split(',').str[0]

# Remove spaces
concatenated_df['Sequence'] = concatenated_df['Sequence'].str.replace(' ', '', regex=False)

# Remove invalid amino acids
concatenated_df['Sequence'] = concatenated_df['Sequence'].apply(lambda x: re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', x))

#from sklearn.model_selection import train_test_split

# Remove sequences with issues
concatenated_df =concatenated_df[~concatenated_df['Sequence'].str.contains('[, ZBOUXJ ]')]
# Optional: show progress bar
concatenated_df

,Sequence,Class
0,FWRRYKKVKKYRRWF,1
1,VDKPPYLPRVRPPRRIYNR,1
2,WWKRWKRIRRIFMMV,1
3,LLCKLKCKLKL,1
4,KLKAGLAKWKAGLAKLKAGLA,1
...,...,...
9047,KSLSTGKSKSFFVRQTNKS,0
9048,KSKSFFVRQTKKS,0
9049,KSLSTGKSKSFFVRQTKKS,0
9050,FSSEKLKARKEKKSRKQAPY,0


In [9]:
concatenated_df = concatenated_df.reset_index(drop=True)
concatenated_

,Sequence,Class
0,FWRRYKKVKKYRRWF,1
1,VDKPPYLPRVRPPRRIYNR,1
2,WWKRWKRIRRIFMMV,1
3,LLCKLKCKLKL,1
4,KLKAGLAKWKAGLAKLKAGLA,1
...,...,...
9041,KSLSTGKSKSFFVRQTNKS,0
9042,KSKSFFVRQTKKS,0
9043,KSLSTGKSKSFFVRQTKKS,0
9044,FSSEKLKARKEKKSRKQAPY,0


In [8]:
import matplotlib.pyplot as plt
import pandas as pd

# Check your DataFrame
print(cleaned_df.columns)  # Debug step

# Assuming your class label column is 'Label'
sequences_class_0 = cleaned_df[cleaned_df['Class'] == 0]['Sequence']
sequences_class_1 = cleaned_df[cleaned_df['Class'] == 1]['Sequence']

# Function to calculate amino acid composition for a given sequence
def calculate_amino_acid_composition(sequence):
    amino_acids = set(sequence)
    composition = {aa: sequence.count(aa) / len(sequence) for aa in amino_acids}
    return composition

# Calculate amino acid composition for each class
composition_class_0 = pd.DataFrame([calculate_amino_acid_composition(seq) for seq in sequences_class_0])
composition_class_1 = pd.DataFrame([calculate_amino_acid_composition(seq) for seq in sequences_class_1])

# Sort amino acids alphabetically
sorted_amino_acids = sorted(set(composition_class_0.columns) | set(composition_class_1.columns))

# Plot amino acid composition
fig, ax = plt.subplots(figsize=(16, 8))
composition_class_0[sorted_amino_acids].mean().plot(kind='bar', color='blue', position=0, width=0.4, label='Class 0')
composition_class_1[sorted_amino_acids].mean().plot(kind='bar', color='orange', position=1, width=0.4, label='Class 1')

ax.set_title('Amino Acid Composition for Class 0 and Class 1')
ax.legend()
plt.show()

NameError: name 'cleaned_df' is not defined

In [11]:
import requests
import os
import time
import pandas as pd

# Configuration
OUTPUT_DIR = "../hd/"
SLEEP_TIME = 5  # Seconds between requests
MAX_RETRIES = 3  # Retry attempts for failed requests
START_INDEX = 0  # <-- Change here to resume

# Create the output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_pdb_from_sequence(seq, out_file_path):
    """Get PDB structure from ESM Atlas API for a given amino acid sequence"""
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"
    headers = {
        "Content-Type": "text/plain"
    }

    # Sanitize sequence
    clean_seq = ''.join(filter(str.isalpha, seq)).upper()

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(url, data=clean_seq, headers=headers, timeout=30)
            if response.ok:
                with open(out_file_path, "w") as f:
                    f.write(response.text)
                return True
            else:
                print(f"Attempt {attempt + 1} failed for {clean_seq[:10]}... "
                      f"Status: {response.status_code}, Response: {response.text[:100]}")
                if response.status_code == 429:
                    time.sleep(SLEEP_TIME * (attempt + 2))  # Exponential backoff
        except Exception as e:
            print(f"Attempt {attempt + 1} error: {str(e)}")
            time.sleep(SLEEP_TIME)

    return False

# Initialize PDB_File column if missing
if 'PDB_File' not in concatenated_df.columns:
    concatenated_df['PDB_File'] = [None] * len(concatenated_df)

# Process each sequence from START_INDEX onward
for idx in range(START_INDEX, len(concatenated_df)):
    seq = concatenated_df.at[idx, 'Sequence']
    filename = f"peptide_{idx}.pdb"
    file_path = os.path.join(OUTPUT_DIR, filename)

    print(f"Processing sequence {idx + 1}/{len(concatenated_df)}: {seq[:15]}...")

    # Skip if already exists (optional)
    if os.path.exists(file_path):
        print(f"Already exists: {file_path}")
        concatenated_df.at[idx, 'PDB_File'] = file_path
        continue

    success = get_pdb_from_sequence(seq, file_path)
    if success:
        print(f"Successfully saved: {file_path}")
        concatenated_df.at[idx, 'PDB_File'] = file_path
    else:
        print(f"Failed to get PDB for index {idx}")
        concatenated_df.at[idx, 'PDB_File'] = None

    time.sleep(SLEEP_TIME)

# Optional: Save the updated DataFrame
# concatenated_df.to_csv("peptides_with_pdb_paths.csv", index=False)
# print("✅ Processing complete. Dataset updated.")


Processing sequence 1/9046: FWRRYKKVKKYRRWF...
Already exists: ../hd/peptide_0.pdb
Processing sequence 2/9046: VDKPPYLPRVRPPRR...
Already exists: ../hd/peptide_1.pdb
Processing sequence 3/9046: WWKRWKRIRRIFMMV...
Already exists: ../hd/peptide_2.pdb
Processing sequence 4/9046: LLCKLKCKLKL...
Already exists: ../hd/peptide_3.pdb
Processing sequence 5/9046: KLKAGLAKWKAGLAK...
Already exists: ../hd/peptide_4.pdb
Processing sequence 6/9046: KLALKLALKAWKAAL...
Already exists: ../hd/peptide_5.pdb
Processing sequence 7/9046: KKMKKALQYIKLLKE...
Already exists: ../hd/peptide_6.pdb
Processing sequence 8/9046: LKVVFKVLFK...
Already exists: ../hd/peptide_7.pdb
Processing sequence 9/9046: KWKSFLKTFKSAKKT...
Already exists: ../hd/peptide_8.pdb
Processing sequence 10/9046: KTVGKLALKHWL...
Already exists: ../hd/peptide_9.pdb
Processing sequence 11/9046: FAKLLAKLAK...
Already exists: ../hd/peptide_10.pdb
Processing sequence 12/9046: KLKLCKKLKLLKLKL...
Already exists: ../hd/peptide_11.pdb
Processing seque

Already exists: ../hd/peptide_892.pdb
Processing sequence 894/9046: RWRPKRWGRWLRKIR...
Already exists: ../hd/peptide_893.pdb
Processing sequence 895/9046: RLPRPRPRPLPFPRP...
Already exists: ../hd/peptide_894.pdb
Processing sequence 896/9046: WRWAKWGLKLWKWKK...
Already exists: ../hd/peptide_895.pdb
Processing sequence 897/9046: RFRWPIRRPPIRPPW...
Already exists: ../hd/peptide_896.pdb
Processing sequence 898/9046: KKLFKKGLKFL...
Already exists: ../hd/peptide_897.pdb
Processing sequence 899/9046: GLPALILWIKRKRQQ...
Already exists: ../hd/peptide_898.pdb
Processing sequence 900/9046: YRKKWFW...
Already exists: ../hd/peptide_899.pdb
Processing sequence 901/9046: RIGSILGRLAKGLPT...
Already exists: ../hd/peptide_900.pdb
Processing sequence 902/9046: RGLRRLGRKIAHGVK...
Already exists: ../hd/peptide_901.pdb
Processing sequence 903/9046: FLFRVASKVFPALIG...
Already exists: ../hd/peptide_902.pdb
Processing sequence 904/9046: RFIYMKGFGKPRFGK...
Already exists: ../hd/peptide_903.pdb
Processing sequen

Already exists: ../hd/peptide_1876.pdb
Processing sequence 1878/9046: RGLFKKLLRKIKKGF...
Already exists: ../hd/peptide_1877.pdb
Processing sequence 1879/9046: LLKKKFKKLQ...
Already exists: ../hd/peptide_1878.pdb
Processing sequence 1880/9046: KWLLKLKLKLLK...
Already exists: ../hd/peptide_1879.pdb
Processing sequence 1881/9046: LWWRRRWWL...
Already exists: ../hd/peptide_1880.pdb
Processing sequence 1882/9046: KKIGAIKKWGAIKKI...
Already exists: ../hd/peptide_1881.pdb
Processing sequence 1883/9046: KWKLFKKIPLKKF...
Already exists: ../hd/peptide_1882.pdb
Processing sequence 1884/9046: ASVVKKLTKKVAKLL...
Already exists: ../hd/peptide_1883.pdb
Processing sequence 1885/9046: WWRRTVAKVRNAGRK...
Already exists: ../hd/peptide_1884.pdb
Processing sequence 1886/9046: GKIIKLKASLKLL...
Already exists: ../hd/peptide_1885.pdb
Processing sequence 1887/9046: RWWRWRKWW...
Already exists: ../hd/peptide_1886.pdb
Processing sequence 1888/9046: KWRRWVRWI...
Already exists: ../hd/peptide_1887.pdb
Processing s

Already exists: ../hd/peptide_2955.pdb
Processing sequence 2957/9046: FPVGRVHRLLRK...
Already exists: ../hd/peptide_2956.pdb
Processing sequence 2958/9046: VDKPPYLPRPRPPRR...
Already exists: ../hd/peptide_2957.pdb
Processing sequence 2959/9046: WKLFKKILKYL...
Already exists: ../hd/peptide_2958.pdb
Processing sequence 2960/9046: RFRPPIRRPPIRPPF...
Already exists: ../hd/peptide_2959.pdb
Processing sequence 2961/9046: FLKHIKSFWRGAKAI...
Already exists: ../hd/peptide_2960.pdb
Processing sequence 2962/9046: GFFKRWKTFF...
Already exists: ../hd/peptide_2961.pdb
Processing sequence 2963/9046: KLKKKLKKFQ...
Already exists: ../hd/peptide_2962.pdb
Processing sequence 2964/9046: GILKTIKSIASKVAN...
Already exists: ../hd/peptide_2963.pdb
Processing sequence 2965/9046: LLLLKFKKLQ...
Already exists: ../hd/peptide_2964.pdb
Processing sequence 2966/9046: NNCKHKKGSGC...
Already exists: ../hd/peptide_2965.pdb
Processing sequence 2967/9046: KRIVQRIKDFKR...
Already exists: ../hd/peptide_2966.pdb
Processing 

Already exists: ../hd/peptide_4126.pdb
Processing sequence 4128/9046: FRRWWKWFK...
Already exists: ../hd/peptide_4127.pdb
Processing sequence 4129/9046: FAKKLAKKLKKLAKK...
Already exists: ../hd/peptide_4128.pdb
Processing sequence 4130/9046: FKKLKKLFSKLWNWK...
Already exists: ../hd/peptide_4129.pdb
Processing sequence 4131/9046: YCKFKVKPKFKRWKL...
Already exists: ../hd/peptide_4130.pdb
Processing sequence 4132/9046: KWRRWIRLL...
Already exists: ../hd/peptide_4131.pdb
Processing sequence 4133/9046: AAHCLAMRRK...
Already exists: ../hd/peptide_4132.pdb
Processing sequence 4134/9046: RWRWPIRRPPIRPPF...
Already exists: ../hd/peptide_4133.pdb
Processing sequence 4135/9046: SPRRRRSQSPRRRR...
Already exists: ../hd/peptide_4134.pdb
Processing sequence 4136/9046: WRTWRTWRTWRTWRT...
Already exists: ../hd/peptide_4135.pdb
Processing sequence 4137/9046: RWCVYAYVRVRGVLR...
Already exists: ../hd/peptide_4136.pdb
Processing sequence 4138/9046: GWCRCVCRRGICRCF...
Already exists: ../hd/peptide_4137.pdb


Already exists: ../hd/peptide_5126.pdb
Processing sequence 5128/9046: SETKKTCVIGAKYAV...
Already exists: ../hd/peptide_5127.pdb
Processing sequence 5129/9046: ACEVCGQRVRQKKKS...
Already exists: ../hd/peptide_5128.pdb
Processing sequence 5130/9046: KQNKKKNKGKSVLFL...
Already exists: ../hd/peptide_5129.pdb
Processing sequence 5131/9046: FKHRLINL...
Already exists: ../hd/peptide_5130.pdb
Processing sequence 5132/9046: KKIKLTRVSKGPL...
Already exists: ../hd/peptide_5131.pdb
Processing sequence 5133/9046: VKISSVTPHKGIKGK...
Already exists: ../hd/peptide_5132.pdb
Processing sequence 5134/9046: RATRRSAKSHISVRI...
Already exists: ../hd/peptide_5133.pdb
Processing sequence 5135/9046: GSPKKKSHSGSRKLC...
Already exists: ../hd/peptide_5134.pdb
Processing sequence 5136/9046: LCGHQKTRATRL...
Already exists: ../hd/peptide_5135.pdb
Processing sequence 5137/9046: NKCVKKRQ...
Already exists: ../hd/peptide_5136.pdb
Processing sequence 5138/9046: LRYLVQKRKIKN...
Already exists: ../hd/peptide_5137.pdb
Proc

Already exists: ../hd/peptide_6293.pdb
Processing sequence 6295/9046: RRRQGGRPRTAHGQD...
Already exists: ../hd/peptide_6294.pdb
Processing sequence 6296/9046: SRSYIRPKCL...
Already exists: ../hd/peptide_6295.pdb
Processing sequence 6297/9046: TTKKKKKKNKS...
Already exists: ../hd/peptide_6296.pdb
Processing sequence 6298/9046: SGEPVVKLHRRKKMV...
Already exists: ../hd/peptide_6297.pdb
Processing sequence 6299/9046: VVGWYRQARRWNR...
Already exists: ../hd/peptide_6298.pdb
Processing sequence 6300/9046: RDLLKKTLQKM...
Already exists: ../hd/peptide_6299.pdb
Processing sequence 6301/9046: CRKKPKTH...
Already exists: ../hd/peptide_6300.pdb
Processing sequence 6302/9046: VAADRKKLPAK...
Already exists: ../hd/peptide_6301.pdb
Processing sequence 6303/9046: SGRQKIASSQRKEKE...
Already exists: ../hd/peptide_6302.pdb
Processing sequence 6304/9046: PGRGIKSSKWTIFGK...
Already exists: ../hd/peptide_6303.pdb
Processing sequence 6305/9046: KNKSFLSRPRFACPK...
Already exists: ../hd/peptide_6304.pdb
Processi

Already exists: ../hd/peptide_7388.pdb
Processing sequence 7390/9046: VPNKQKFRGSRPCPL...
Already exists: ../hd/peptide_7389.pdb
Processing sequence 7391/9046: RGIKRSTSAGAVRKP...
Already exists: ../hd/peptide_7390.pdb
Processing sequence 7392/9046: YKLGKNKWFFQKLRF...
Already exists: ../hd/peptide_7391.pdb
Processing sequence 7393/9046: HLSDKKVGSKATRHY...
Already exists: ../hd/peptide_7392.pdb
Processing sequence 7394/9046: STHAKKTRKLRGHVS...
Already exists: ../hd/peptide_7393.pdb
Processing sequence 7395/9046: KRNKPKVKKFIKPDH...
Already exists: ../hd/peptide_7394.pdb
Processing sequence 7396/9046: KKKRSKSADQ...
Already exists: ../hd/peptide_7395.pdb
Processing sequence 7397/9046: KKKKKNKKGVVTG...
Already exists: ../hd/peptide_7396.pdb
Processing sequence 7398/9046: HMIKNKPQLRF...
Already exists: ../hd/peptide_7397.pdb
Processing sequence 7399/9046: AVTGKRAAIRAKTA...
Already exists: ../hd/peptide_7398.pdb
Processing sequence 7400/9046: WKLKGMKVMKRLT...
Already exists: ../hd/peptide_7399.

Already exists: ../hd/peptide_8338.pdb
Processing sequence 8340/9046: SGKKRHSCYVYGIAR...
Already exists: ../hd/peptide_8339.pdb
Processing sequence 8341/9046: LGKQDYKKTKPILRA...
Already exists: ../hd/peptide_8340.pdb
Processing sequence 8342/9046: GCRNRKRTCRRH...
Already exists: ../hd/peptide_8341.pdb
Processing sequence 8343/9046: NTKVARKLAMV...
Already exists: ../hd/peptide_8342.pdb
Processing sequence 8344/9046: TGATAKAKSALGKKV...
Already exists: ../hd/peptide_8343.pdb
Processing sequence 8345/9046: AKTGGAATKTKGMPG...
Already exists: ../hd/peptide_8344.pdb
Processing sequence 8346/9046: KRKEKKKEKEKEKEK...
Already exists: ../hd/peptide_8345.pdb
Processing sequence 8347/9046: QKQSKKKKEGQQQQQ...
Already exists: ../hd/peptide_8346.pdb
Processing sequence 8348/9046: REKTVKMKMKTVKIK...
Already exists: ../hd/peptide_8347.pdb
Processing sequence 8349/9046: IRNKQNEFLRMMRKK...
Already exists: ../hd/peptide_8348.pdb
Processing sequence 8350/9046: GSPKSRRLSFYKIK...
Already exists: ../hd/peptide

In [12]:
import os

OUTPUT_DIR = "../hd//"
existing_files = os.listdir(OUTPUT_DIR)

for file in existing_files:
    if file.endswith(".pdb") and file.startswith("peptide_"):
        try:
            idx = int(file.replace("peptide_", "").replace(".pdb", ""))
            file_path = os.path.join(OUTPUT_DIR, file)
            if 0 <= idx < len(concatenated_df):
                concatenated_df.at[idx, 'PDB_File'] = file_path
        except ValueError:
            continue  # In case there's a filename that doesn't match the pattern

# Optional: Save the updated DataFrame
concatenated_df.to_csv("peptides_with_pdb_paths_fixed.csv", index=False)
print("✅ PDB_File column updated from existing files.")
concatenated_df

✅ PDB_File column updated from existing files.


,Sequence,Class,PDB_File
0,FWRRYKKVKKYRRWF,1,../hd//peptide_0.pdb
1,VDKPPYLPRVRPPRRIYNR,1,../hd//peptide_1.pdb
2,WWKRWKRIRRIFMMV,1,../hd//peptide_2.pdb
3,LLCKLKCKLKL,1,../hd//peptide_3.pdb
4,KLKAGLAKWKAGLAKLKAGLA,1,../hd//peptide_4.pdb
...,...,...,...
9041,KSLSTGKSKSFFVRQTNKS,0,../hd//peptide_9041.pdb
9042,KSKSFFVRQTKKS,0,../hd//peptide_9042.pdb
9043,KSLSTGKSKSFFVRQTKKS,0,../hd//peptide_9043.pdb
9044,FSSEKLKARKEKKSRKQAPY,0,../hd//peptide_9044.pdb


In [13]:
from Bio.PDB import PDBParser, DSSP
import pandas as pd
import numpy as np
import os

# Path to folder containing PDB files
pdb_directory = "../hd/./"  # Adjust if needed

# Initialize DSSP feature columns
dssp_columns = ['Mean_RSA', 'Mean_Phi', 'Mean_Psi']
for col in dssp_columns:
    concatenated_df[col] = np.nan

parser = PDBParser(QUIET=True)

# Loop to calculate RSA, Phi, Psi features
for idx in concatenated_df.index:
    pdb_filename = f"peptide_{idx}.pdb"
    pdb_path = os.path.join(pdb_directory, pdb_filename)

    if not os.path.exists(pdb_path):
        print(f"File not found: {pdb_path}")
        continue

    try:
        structure = parser.get_structure(f"pep_{idx}", pdb_path)
        model = structure[0]
        dssp = DSSP(model, pdb_path)

        rsa_list = []
        phi_list = []
        psi_list = []

        for res in dssp:
            rsa = res[3]
            phi = res[4]
            psi = res[5]

            rsa_list.append(rsa)
            phi_list.append(phi)
            psi_list.append(psi)

        # Compute means
        concatenated_df.at[idx, 'Mean_RSA'] = np.mean(rsa_list) if rsa_list else np.nan
        concatenated_df.at[idx, 'Mean_Phi'] = np.mean(phi_list) if phi_list else np.nan
        concatenated_df.at[idx, 'Mean_Psi'] = np.mean(psi_list) if psi_list else np.nan

    except Exception as e:
        print(f"Error processing index {idx} - file: {pdb_path}: {e}")


In [14]:
from Bio.PDB import PDBParser, DSSP
import os

# Columns to store DSSP secondary structure percentages
helices = []
sheets = []
coils = []

parser = PDBParser()

for idx in concatenated_df.index:
    pdb_file = f"../hd/peptide_{idx}.pdb"
    try:
        structure = parser.get_structure(f"pep_{idx}", pdb_file)
        model = structure[0]
        dssp = DSSP(model, pdb_file)
        
        ss_list = [dssp[key][2] for key in dssp.keys()]
        total = len(ss_list)
        
        helix_count = sum(s in "HGI" for s in ss_list)
        sheet_count = sum(s in "BE" for s in ss_list)
        coil_count  = sum(s in "ST " for s in ss_list)  # includes turn, bend, and loops
        
        helices.append(helix_count / total * 100)
        sheets.append(sheet_count / total * 100)
        coils.append(coil_count / total * 100)
    except Exception as e:
        print(f"Failed to process {pdb_file}: {e}")
        helices.append(None)
        sheets.append(None)
        coils.append(None)

# Add DSSP % columns to DataFrame
concatenated_df['%Helix'] = helices
concatenated_df['%Sheet'] = sheets
concatenated_df['%Coil'] = coils


In [15]:
import mdtraj as md
import pandas as pd
import numpy as np
import os

# Folder containing your PDBs
pdb_folder = '../hd/'

# Initialize columns
concatenated_df['RoG'] = np.nan
concatenated_df['SASA'] = np.nan
concatenated_df['Compactness'] = np.nan

# Loop through each peptide
for idx in concatenated_df.index:
    pdb_path = os.path.join(pdb_folder, f"peptide_{idx}.pdb")

    if not os.path.exists(pdb_path):
        print(f"Missing: {pdb_path}")
        continue

    try:
        traj = md.load(pdb_path)
        
        # Radius of Gyration (RoG)
        rog = md.compute_rg(traj)[0]
        concatenated_df.at[idx, 'RoG'] = rog

        # Solvent Accessible Surface Area (SASA)
        sasa = md.shrake_rupley(traj)[0].sum()
        concatenated_df.at[idx, 'SASA'] = sasa

        # Compactness = Mass / SASA (or optionally 1/SASA per atom)
        n_atoms = traj.n_atoms
        compactness = n_atoms / sasa if sasa > 0 else np.nan
        concatenated_df.at[idx, 'Compactness'] = compactness

    except Exception as e:
        print(f"Error at index {idx}: {e}")


/home/hrushikesh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
cleaned_df = concatenated_df
cleaned_df
# Remove sequences that contain "RLYR"

cleaned_df


,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness
0,FWRRYKKVKKYRRWF,1,../hd//peptide_0.pdb,0.859582,-52.913333,47.360000,46.666667,0.0,20.000000,1.094001,27.651440,5.822482
1,VDKPPYLPRVRPPRRIYNR,1,../hd//peptide_1.pdb,0.865338,-25.389474,113.342105,0.000000,0.0,0.000000,1.451317,30.585161,5.525555
2,WWKRWKRIRRIFMMV,1,../hd//peptide_2.pdb,0.716046,-42.386667,-8.420000,80.000000,0.0,6.666667,0.808917,22.301994,6.905212
3,LLCKLKCKLKL,1,../hd//peptide_3.pdb,0.829469,-41.363636,32.109091,45.454545,0.0,18.181818,0.697707,16.447903,5.350226
4,KLKAGLAKWKAGLAKLKAGLA,1,../hd//peptide_4.pdb,0.702707,-41.447619,-23.457143,90.476190,0.0,0.000000,0.989981,22.909313,6.547556
...,...,...,...,...,...,...,...,...,...,...,...,...
9041,KSLSTGKSKSFFVRQTNKS,0,../hd//peptide_9041.pdb,0.903197,-33.236842,107.900000,0.000000,0.0,0.000000,1.584088,28.541864,5.220402
9042,KSKSFFVRQTKKS,0,../hd//peptide_9042.pdb,0.899669,-17.700000,103.261538,0.000000,0.0,0.000000,1.071776,20.900116,5.263129
9043,KSLSTGKSKSFFVRQTKKS,0,../hd//peptide_9043.pdb,0.887751,-26.436842,104.094737,0.000000,0.0,0.000000,1.553086,28.279654,5.304167
9044,FSSEKLKARKEKKSRKQAPY,0,../hd//peptide_9044.pdb,0.701268,-43.470000,14.855000,65.000000,0.0,15.000000,1.004550,25.511311,6.624513


In [17]:
import pandas as pd
import peptides

# Assuming you have a DataFrame named 'df' with a 'Sequence' column

# Define the peptide descriptors you want to calculate
descriptors = [
    'frequencies',
    'aliphatic_index',
    'boman',
    'charge',
    'isoelectric_point',
    'hydrophobic_moment',
    'hydrophobicity',
    'instability_index',
    'mass_shift',
    'molecular_weight',
    'mz',
    'structural_class'
]

# Create a dictionary to store descriptor values
descriptor_values_dict = {}

# Iterate through each descriptor and calculate values for each sequence
for descriptor in descriptors:
    descriptor_values = []
    for sequence in cleaned_df['Sequence']:
        peptide = peptides.Peptide(sequence)
        value = getattr(peptide, descriptor)()
        descriptor_values.append(value)
    descriptor_values_dict[descriptor] = descriptor_values

# Create a new DataFrame using the descriptor values dictionary
descriptors = pd.DataFrame(descriptor_values_dict)

In [18]:
import pandas as pd

# Assuming you have a DataFrame named 'descriptors_df' with a 'frequencies' column
# containing dictionaries of amino acid frequencies

# Expand 'frequencies' column into separate columns
amino_acid_frequencies = pd.DataFrame(descriptors['frequencies'].tolist()).fillna(0)

# Combine the expanded DataFrame with the original DataFrame
descriptors_df = pd.concat([descriptors, amino_acid_frequencies], axis=1)

# Drop the original 'frequencies' column
descriptors_df.drop(columns=['frequencies'], inplace=True)

# Display the resulting DataFrame
descriptors_df


,aliphatic_index,boman,charge,isoelectric_point,hydrophobic_moment,hydrophobicity,instability_index,mass_shift,molecular_weight,mz,...,T,W,Y,V,O,U,B,Z,J,X
0,19.333333,4.500000,7.995090,12.130605,0.630528,-1.880000,100.013333,48.161032,2247.72584,1124.149951,...,0.000000,0.133333,0.133333,0.066667,0.0,0.0,0.0,0.0,0.0,0.0
1,71.578947,4.098421,4.996418,11.947735,0.606835,-1.436842,55.884211,36.120774,2392.83874,1196.695446,...,0.000000,0.000000,0.105263,0.105263,0.0,0.0,0.0,0.0,0.0,0.0
2,71.333333,2.815333,5.997381,12.984698,0.750688,-0.580000,127.253333,36.120774,2192.76624,1096.621661,...,0.000000,0.200000,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0
3,177.272727,-0.450909,3.872855,10.348813,0.279752,0.763636,-29.500000,24.080516,1302.78624,708.943280,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,121.428571,-0.348095,5.996214,11.499626,0.357415,0.204762,-6.647619,36.120774,2136.69854,1068.690761,...,0.000000,0.047619,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9041,35.789474,2.925263,4.996801,11.997381,0.345357,-1.015789,15.226316,30.100645,2130.43194,1065.586916,...,0.105263,0.000000,0.000000,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
9042,22.307692,3.494615,4.996801,11.997381,0.390433,-1.300000,24.046154,30.100645,1570.85534,785.956821,...,0.076923,0.000000,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0
9043,35.789474,2.867895,5.996506,12.054553,0.390433,-1.036842,10.673684,36.120774,2144.50224,1072.612931,...,0.105263,0.000000,0.000000,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
9044,29.500000,4.056000,5.998907,11.106017,0.396581,-1.900000,51.370000,48.161032,2409.81744,1205.189861,...,0.000000,0.000000,0.050000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
import pandas as pd
from peptides import Peptide

# Function to calculate Kidera factors for a given peptide sequence
def calculate_kidera_factors(peptide_sequence):
    peptide = Peptide(peptide_sequence)
    return peptide.kidera_factors()

# Assuming concatenated_df has a 'Sequence' column with peptide sequences
# Calculate Kidera factors for each peptide and create a new DataFrame
kidera_factors_df = cleaned_df['Sequence'].apply(calculate_kidera_factors).apply(pd.Series)

# Rename the Kidera factors columns
kidera_factors_df.columns = [f'KF{i+1}' for i in range(kidera_factors_df.shape[1])]

# Concatenate the original DataFrame with the new Kidera factors DataFrame
descriptors3_df = pd.concat([kidera_factors_df], axis=1)

# Print the updated DataFrame with renamed Kidera factors columns
descriptors3_df

,KF1,KF2,KF3,KF4,KF5,KF6,KF7,KF8,KF9,KF10
0,0.114667,1.118000,0.402667,0.450667,-0.134667,-0.486000,0.464667,0.109333,-0.182667,0.383333
1,0.646842,0.347895,0.336842,0.270000,-0.178421,-0.318421,0.158421,-0.521053,0.332632,0.227895
2,-0.274000,0.888667,0.485333,0.089333,-0.237333,0.119333,0.200000,-0.034667,-0.460667,0.020667
3,-0.574545,0.136364,-0.110909,-0.072727,0.180909,-1.082727,1.130909,-0.500000,0.235455,0.840909
4,-0.567619,-0.422857,-0.467143,0.049048,0.002381,-1.162381,0.665714,0.091429,-0.316667,0.292857
...,...,...,...,...,...,...,...,...,...,...
9041,0.176842,-0.146842,0.235263,0.495789,0.262105,-0.598947,-0.007895,-0.230526,-0.254737,-0.017895
9042,-0.006154,0.161538,0.271538,0.646154,0.443846,-0.687692,0.092308,-0.289231,-0.171538,0.020769
9043,0.098947,-0.100000,0.229474,0.542632,0.333684,-0.703684,0.057368,-0.299474,-0.337895,0.104737
9044,-0.173500,0.181500,-0.210000,0.713000,0.119000,-0.712500,0.260500,-0.287500,-0.139500,0.099000


In [20]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Assuming you have a DataFrame named 'pos' with a 'Sequence' column
# Create a new DataFrame to store the results
d4 = pd.DataFrame()

def calculate_tpsa(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate TPSA using AllChem method
        tpsa = AllChem.CalcTPSA(peptide_molecule)

        return tpsa
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create a new 'TPSA' column
#new_pos['Sequence'] = cleaned_df['Sequence']
d4['TPSA'] = cleaned_df['Sequence'].apply(calculate_tpsa)

# Display the new DataFrame with TPSA values
d4

,TPSA
0,894.44
1,999.54
2,817.73
3,458.40
4,817.23
...,...
9041,980.89
9042,702.51
9043,963.82
9044,1085.96


In [21]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Crippen

# Assuming you have a DataFrame named 'cleaned_df' with a 'Sequence' column
# Create a new DataFrame to store the results
d5 = pd.DataFrame()

def calculate_aromaticity(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate aromaticity using Crippen method
        aromaticity = Crippen.MolLogP(peptide_molecule)

        return aromaticity
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create a new 'Aromaticity' column
d5['Aromaticity'] = cleaned_df['Sequence'].apply(calculate_aromaticity)

# Display the new DataFrame with Aromaticity values
d5

,Aromaticity
0,-1.75942
1,-7.24805
2,0.16568
3,-0.14760
4,-4.16360
...,...
9041,-14.41313
9042,-8.24023
9043,-13.15953
9044,-11.33486


In [22]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Crippen, Lipinski, Descriptors

# Assuming you have a DataFrame named 'cleaned_df' with a 'Sequence' column
# Create a new DataFrame to store the results
d6 = pd.DataFrame()

def calculate_descriptors(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate various molecular descriptors
        heavy_atom_count = Descriptors.HeavyAtomCount(peptide_molecule)
        logp = Crippen.MolLogP(peptide_molecule)
        mol_logp = Crippen.MolLogP(peptide_molecule)
        # MolLogS calculation is not directly available in RDKit, consider using other methods/tools
        fraction_csp3 = Lipinski.FractionCSP3(peptide_molecule)
        bertz_ct = Descriptors.BertzCT(peptide_molecule)

        return heavy_atom_count, logp, mol_logp, fraction_csp3, bertz_ct
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create new columns for descriptors
#new_pos['Sequence'] = cleaned_df['Sequence']
d6[['HeavyAtomCount', 'LogP', 'MolLogP', 'FractionCSP3', 'BertzCT']] = cleaned_df['Sequence'].apply(calculate_descriptors).apply(pd.Series)

# Display the new DataFrame with descriptor values
d6

,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT
0,162.0,-1.75942,-1.75942,0.468468,6167.457349
1,170.0,-7.24805,-7.24805,0.651376,5790.862434
2,155.0,0.16568,0.16568,0.533333,5810.258002
3,89.0,-0.14760,-0.14760,0.816667,2181.007600
4,151.0,-4.16360,-4.16360,0.712871,4692.193883
...,...,...,...,...,...
9041,150.0,-14.41313,-14.41313,0.634409,4705.308666
9042,111.0,-8.24023,-8.24023,0.619718,3290.838106
9043,151.0,-13.15953,-13.15953,0.652632,4650.711363
9044,170.0,-11.33486,-11.33486,0.654206,5415.940777


In [23]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors

# Assuming you have a DataFrame named 'cleaned_df' with a 'Sequence' column
# Create a new DataFrame to store the results
d7 = pd.DataFrame()

def calculate_polarity(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate descriptors related to polarity
        num_h_acceptors = Lipinski.NumHAcceptors(peptide_molecule)
        num_h_donors = Lipinski.NumHDonors(peptide_molecule)
        #polar_surface_area = Descriptors.TPSA(peptide_molecule)

        return num_h_acceptors, num_h_donors
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create new columns for polarity descriptors
#new_pos['Sequence'] = cleaned_df['Sequence']
d7[['NumHAcceptors', 'NumHDonors',]] = cleaned_df['Sequence'].apply(calculate_polarity).apply(pd.Series)

# Display the new DataFrame with polarity descriptors
d7

,NumHAcceptors,NumHDonors
0,26,36
1,30,35
2,24,33
3,18,18
4,28,29
...,...,...
9041,34,36
9042,24,26
9043,34,36
9044,36,39


In [24]:
from PyBioMed.PyProtein import PseudoAAC
from PyBioMed.PyProtein.PseudoAAC import GetAPseudoAAC1
import pandas as pd

# Assuming you have a DataFrame named 'cleaned_df' with a 'Sequence' column
# Example DataFrame structure:
# cleaned_df = pd.DataFrame({'Sequence': ['RACRACRACRAC', 'ACDEFGHIKLM', ...]})

# Create a DataFrame to store the results
result_df = pd.DataFrame()

# Calculate PseudoAAC (APAAC) values for each sequence in the 'Sequence' column
for sequence in cleaned_df['Sequence']:
    apaac_values = GetAPseudoAAC1(sequence, 2, 0.5)
    result_df = pd.concat([result_df, pd.DataFrame([apaac_values], columns=[f'APAAC{i}' for i in range(1, len(apaac_values)+1)])], ignore_index=True)

# Display the DataFrame with each APAAC value in separate columns for each sequence
print(result_df)

      APAAC1  APAAC2  APAAC3  APAAC4  APAAC5  APAAC6  APAAC7  APAAC8  APAAC9  \
0      0.000  37.244   0.000   0.000    0.00   0.000   0.000   0.000     0.0   
1      0.000  32.895   6.579   6.579    0.00   0.000   0.000   0.000     0.0   
2      0.000  52.754   0.000   0.000    0.00   0.000   0.000   0.000     0.0   
3      0.000   0.000   0.000   0.000   15.97   0.000   0.000   0.000     0.0   
4     33.812   0.000   0.000   0.000    0.00   0.000   0.000  16.907     0.0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
9041   0.000   5.102   5.102   0.000    0.00   0.000   5.102   5.102     0.0   
9042   0.000   5.366   0.000   0.000    0.00   0.000   5.366   0.000     0.0   
9043   0.000   4.477   0.000   0.000    0.00   0.000   4.477   4.477     0.0   
9044   3.810   3.810   0.000   0.000    0.00   3.810   1.905   0.000     0.0   
9045   3.805   3.805   0.000   1.903    0.00   1.903   1.903   0.000     0.0   

      APAAC10  APAAC11  APAAC12  APAAC1

In [25]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors

# Assuming you have a DataFrame named 'cleaned_df' with a 'Sequence' column
# Create a new DataFrame to store the results
df8 = pd.DataFrame()

def calculate_amino_acid_properties(peptide_sequence):
    ng_count = peptide_sequence.count('C(N)N')
    npa_count = peptide_sequence.count('C(N)C')
    nncaa_count = peptide_sequence.count('D') + peptide_sequence.count('E')

    return ng_count, npa_count, nncaa_count
# Apply the function to the 'Sequence' column and create new columns for amino acid properties
df8[['NG', 'NPA', 'NNCAA']] = cleaned_df['Sequence'].apply(calculate_amino_acid_properties).apply(pd.Series)

# Display the DataFrame with amino acid properties
df8

,NG,NPA,NNCAA
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
9041,0,0,0
9042,0,0,0
9043,0,0,0
9044,0,0,2


In [26]:
cleaned_df.reset_index(drop=True, inplace=True)
descriptors_df.reset_index(drop=True, inplace=True)
#descriptors3_df.reset_index(drop=True, inplace=True)
#descriptors4_df_cleaned.reset_index(drop=True, inplace=True)
#result_df.reset_index(drop=True, inplace=True)
d4.reset_index(drop=True, inplace=True)
d5.reset_index(drop=True, inplace=True)
d6.reset_index(drop=True, inplace=True)
d7.reset_index(drop=True, inplace=True)
df8.reset_index(drop=True, inplace=True)
#df9.reset_index(drop=True, inplace=True)

In [27]:
ll = pd.concat([cleaned_df,descriptors_df,d4,d5,d6,d7,df8], axis=1)
ll = ll.fillna(0)
ll

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,FWRRYKKVKKYRRWF,1,../hd//peptide_0.pdb,0.859582,-52.913333,47.360000,46.666667,0.0,20.000000,1.094001,...,162.0,-1.75942,-1.75942,0.468468,6167.457349,26,36,0,0,0
1,VDKPPYLPRVRPPRRIYNR,1,../hd//peptide_1.pdb,0.865338,-25.389474,113.342105,0.000000,0.0,0.000000,1.451317,...,170.0,-7.24805,-7.24805,0.651376,5790.862434,30,35,0,0,1
2,WWKRWKRIRRIFMMV,1,../hd//peptide_2.pdb,0.716046,-42.386667,-8.420000,80.000000,0.0,6.666667,0.808917,...,155.0,0.16568,0.16568,0.533333,5810.258002,24,33,0,0,0
3,LLCKLKCKLKL,1,../hd//peptide_3.pdb,0.829469,-41.363636,32.109091,45.454545,0.0,18.181818,0.697707,...,89.0,-0.14760,-0.14760,0.816667,2181.007600,18,18,0,0,0
4,KLKAGLAKWKAGLAKLKAGLA,1,../hd//peptide_4.pdb,0.702707,-41.447619,-23.457143,90.476190,0.0,0.000000,0.989981,...,151.0,-4.16360,-4.16360,0.712871,4692.193883,28,29,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9041,KSLSTGKSKSFFVRQTNKS,0,../hd//peptide_9041.pdb,0.903197,-33.236842,107.900000,0.000000,0.0,0.000000,1.584088,...,150.0,-14.41313,-14.41313,0.634409,4705.308666,34,36,0,0,0
9042,KSKSFFVRQTKKS,0,../hd//peptide_9042.pdb,0.899669,-17.700000,103.261538,0.000000,0.0,0.000000,1.071776,...,111.0,-8.24023,-8.24023,0.619718,3290.838106,24,26,0,0,0
9043,KSLSTGKSKSFFVRQTKKS,0,../hd//peptide_9043.pdb,0.887751,-26.436842,104.094737,0.000000,0.0,0.000000,1.553086,...,151.0,-13.15953,-13.15953,0.652632,4650.711363,34,36,0,0,0
9044,FSSEKLKARKEKKSRKQAPY,0,../hd//peptide_9044.pdb,0.701268,-43.470000,14.855000,65.000000,0.0,15.000000,1.004550,...,170.0,-11.33486,-11.33486,0.654206,5415.940777,36,39,0,0,2


In [28]:
from Bio import SeqIO

# Save sequences to a FASTA file for CD-HIT
with open("all_sequences.fasta", "w") as f:
    for i, seq in enumerate(ll["Sequence"]):
        f.write(f">seq_{i}\n{seq}\n")


In [34]:
import subprocess

identity = 0.95  # 90% identity
cmd = f"cd-hit -i all_sequences.fasta -o clustered.fasta -c {identity}"

subprocess.run(cmd, shell=True)


Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i all_sequences.fasta -o clustered.fasta -c
         0.95

Started: Mon Dec  1 21:32:48 2025
                            Output                              
----------------------------------------------------------------
total seq: 7863
longest and shortest : 39 and 11
Total letters: 154910
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 1M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 77M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90339759

comparing sequences from          0  to       7863
.......
     7863  finished       6570  clusters

Approximated maximum memory consumption: 78M
writing new database
writing clustering information
program completed !

Total CPU time 0.09


CompletedProcess(args='cd-hit -i all_sequences.fasta -o clustered.fasta -c 0.95', returncode=0)

In [36]:
representative_ids = []

with open("clustered.fasta") as f:
    for line in f:
        if line.startswith(">"):
            rep_id = line.strip().replace(">", "")
            representative_ids.append(rep_id)


In [37]:
kept_indices = [int(x.split("_")[1]) for x in representative_ids]


In [38]:
ll_filtered = ll.iloc[kept_indices].reset_index(drop=True)
ll_filtered


,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,FWRRYKKVKKYRRWF,1,../hd//peptide_0.pdb,0.859582,-52.913333,47.360000,46.666667,0.0,20.000000,1.094001,...,162.0,-1.75942,-1.75942,0.468468,6167.457349,26,36,0,0,0
1,VDKPPYLPRVRPPRRIYNR,1,../hd//peptide_1.pdb,0.865338,-25.389474,113.342105,0.000000,0.0,0.000000,1.451317,...,170.0,-7.24805,-7.24805,0.651376,5790.862434,30,35,0,0,1
2,WWKRWKRIRRIFMMV,1,../hd//peptide_2.pdb,0.716046,-42.386667,-8.420000,80.000000,0.0,6.666667,0.808917,...,155.0,0.16568,0.16568,0.533333,5810.258002,24,33,0,0,0
3,LLCKLKCKLKL,1,../hd//peptide_3.pdb,0.829469,-41.363636,32.109091,45.454545,0.0,18.181818,0.697707,...,89.0,-0.14760,-0.14760,0.816667,2181.007600,18,18,0,0,0
4,KLKAGLAKWKAGLAKLKAGLA,1,../hd//peptide_4.pdb,0.702707,-41.447619,-23.457143,90.476190,0.0,0.000000,0.989981,...,151.0,-4.16360,-4.16360,0.712871,4692.193883,28,29,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,ALKALSLGKSLSTGKSKSFFVRQTKKS,0,../hd//peptide_9035.pdb,0.903978,-31.392593,93.614815,0.000000,0.0,0.000000,2.131840,...,204.0,-15.35033,-15.35033,0.684615,6540.588614,44,46,0,0,0
6566,RKSKSFFVRQTNKS,0,../hd//peptide_9037.pdb,0.874635,-26.900000,111.635714,0.000000,0.0,0.000000,1.183591,...,121.0,-10.74576,-10.74576,0.600000,3721.101187,26,30,0,0,0
6567,KSLSTGKSKSFFVRQTNKS,0,../hd//peptide_9041.pdb,0.903197,-33.236842,107.900000,0.000000,0.0,0.000000,1.584088,...,150.0,-14.41313,-14.41313,0.634409,4705.308666,34,36,0,0,0
6568,FSSEKLKARKEKKSRKQAPY,0,../hd//peptide_9044.pdb,0.701268,-43.470000,14.855000,65.000000,0.0,15.000000,1.004550,...,170.0,-11.33486,-11.33486,0.654206,5415.940777,36,39,0,0,2


In [39]:
categorical_df = ll_filtered.copy()

# Convert the "structural_class" column to categorical
categorical_df['structural_class'] = categorical_df['structural_class'].astype('category')

# Assign categorical codes to the categories
categorical_df['structural_class'] = categorical_df['structural_class'].cat.codes
categorical_df
categorical_df.to_csv('High-charge-density.csv', index=False)

In [46]:
# Then drop unnecessary columns
X = categorical_df.drop(['Class','Sequence','PDB_File','B','Z','X','O','J','U'], axis=1)
X = X.fillna(0)
y = categorical_df['Class']
y=y.astype('int')

X.columns

Index(['Mean_RSA', 'Mean_Phi', 'Mean_Psi', '%Helix', '%Sheet', '%Coil', 'RoG',
       'SASA', 'Compactness', 'aliphatic_index', 'boman', 'charge',
       'isoelectric_point', 'hydrophobic_moment', 'hydrophobicity',
       'instability_index', 'mass_shift', 'molecular_weight', 'mz',
       'structural_class', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I',
       'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'TPSA', 'Aromaticity',
       'HeavyAtomCount', 'LogP', 'MolLogP', 'FractionCSP3', 'BertzCT',
       'NumHAcceptors', 'NumHDonors', 'NG', 'NPA', 'NNCAA'],
      dtype='object')

In [47]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assume X is your DataFrame
scaler = MinMaxScaler()

# Apply Min-Max Normalization to all columns in X
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Display the normalized DataFrame
print("Normalized DataFrame (Min-Max Scaling):")
X_normalized

Normalized DataFrame (Min-Max Scaling):


,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness,aliphatic_index,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,0.730351,0.247142,0.403525,0.493333,0.0,0.307692,0.188426,0.290346,0.310168,0.068162,...,0.307971,0.802631,0.802631,0.333725,0.346596,0.207547,0.277108,0.0,0.0,0.000000
1,0.741743,0.433969,0.743110,0.000000,0.0,0.000000,0.321195,0.348826,0.242876,0.252362,...,0.336957,0.683278,0.683278,0.667730,0.315575,0.283019,0.265060,0.0,0.0,0.111111
2,0.446235,0.318595,0.116447,0.845714,0.0,0.102564,0.082496,0.183711,0.555544,0.251496,...,0.282609,0.844493,0.844493,0.452174,0.317173,0.169811,0.240964,0.0,0.0,0.000000
3,0.670746,0.325539,0.325035,0.480519,0.0,0.279720,0.041174,0.067017,0.203142,0.625000,...,0.043478,0.837681,0.837681,0.969565,0.018220,0.056604,0.060241,0.0,0.0,0.000000
4,0.419833,0.324969,0.039057,0.956463,0.0,0.000000,0.149775,0.195817,0.474490,0.428114,...,0.268116,0.750351,0.750351,0.780026,0.225074,0.245283,0.192771,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,0.818228,0.393221,0.641581,0.000000,0.0,0.000000,0.574058,0.510089,0.180167,0.267688,...,0.460145,0.507089,0.507089,0.728428,0.377332,0.547170,0.397590,0.0,0.0,0.000000
6566,0.760146,0.423716,0.734328,0.000000,0.0,0.000000,0.221715,0.184395,0.208171,0.073031,...,0.159420,0.607218,0.607218,0.573913,0.145082,0.207547,0.204819,0.0,0.0,0.000000
6567,0.816681,0.380702,0.715101,0.000000,0.0,0.000000,0.370528,0.308095,0.173720,0.126181,...,0.264493,0.527469,0.527469,0.636746,0.226155,0.358491,0.277108,0.0,0.0,0.000000
6568,0.416984,0.311242,0.236234,0.687143,0.0,0.230769,0.155188,0.247685,0.491930,0.104006,...,0.336957,0.594408,0.594408,0.672897,0.284692,0.396226,0.313253,0.0,0.0,0.222222


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [49]:
X_normalized

,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness,aliphatic_index,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,0.730351,0.247142,0.403525,0.493333,0.0,0.307692,0.188426,0.290346,0.310168,0.068162,...,0.307971,0.802631,0.802631,0.333725,0.346596,0.207547,0.277108,0.0,0.0,0.000000
1,0.741743,0.433969,0.743110,0.000000,0.0,0.000000,0.321195,0.348826,0.242876,0.252362,...,0.336957,0.683278,0.683278,0.667730,0.315575,0.283019,0.265060,0.0,0.0,0.111111
2,0.446235,0.318595,0.116447,0.845714,0.0,0.102564,0.082496,0.183711,0.555544,0.251496,...,0.282609,0.844493,0.844493,0.452174,0.317173,0.169811,0.240964,0.0,0.0,0.000000
3,0.670746,0.325539,0.325035,0.480519,0.0,0.279720,0.041174,0.067017,0.203142,0.625000,...,0.043478,0.837681,0.837681,0.969565,0.018220,0.056604,0.060241,0.0,0.0,0.000000
4,0.419833,0.324969,0.039057,0.956463,0.0,0.000000,0.149775,0.195817,0.474490,0.428114,...,0.268116,0.750351,0.750351,0.780026,0.225074,0.245283,0.192771,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,0.818228,0.393221,0.641581,0.000000,0.0,0.000000,0.574058,0.510089,0.180167,0.267688,...,0.460145,0.507089,0.507089,0.728428,0.377332,0.547170,0.397590,0.0,0.0,0.000000
6566,0.760146,0.423716,0.734328,0.000000,0.0,0.000000,0.221715,0.184395,0.208171,0.073031,...,0.159420,0.607218,0.607218,0.573913,0.145082,0.207547,0.204819,0.0,0.0,0.000000
6567,0.816681,0.380702,0.715101,0.000000,0.0,0.000000,0.370528,0.308095,0.173720,0.126181,...,0.264493,0.527469,0.527469,0.636746,0.226155,0.358491,0.277108,0.0,0.0,0.000000
6568,0.416984,0.311242,0.236234,0.687143,0.0,0.230769,0.155188,0.247685,0.491930,0.104006,...,0.336957,0.594408,0.594408,0.672897,0.284692,0.396226,0.313253,0.0,0.0,0.222222


In [50]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

# Create the XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=60, learning_rate=0.2, max_depth=2,random_state=42)

# Perform 10-fold cross-validation
cv_scores = cross_val_score(xgb_classifier, X_train, y_train, cv=10)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# Fit the model on the entire training set
xgb_classifier.fit(X_train, y_train)

# Predictions on the training set
y_train_pred = xgb_classifier.predict(X_train)

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)

# Predictions on the test set
y_test_pred = xgb_classifier.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Calculate precision on the test set
precision = precision_score(y_test, y_test_pred)
print("Precision:", precision)

# Calculate sensitivity (recall) on the test set
recall = recall_score(y_test, y_test_pred)
print("Sensitivity (Recall):", recall)

# Calculate ROC AUC score on the test set
roc_auc = roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:,1])
print("ROC AUC:", roc_auc)





Cross-Validation Scores: [0.90684411 0.91825095 0.90304183 0.92015209 0.91444867 0.91634981
 0.91809524 0.89904762 0.90285714 0.92      ]
Mean CV Accuracy: 0.9119087452471483
Train Accuracy: 0.9233257229832572
Test Accuracy: 0.9117199391171994
Precision: 0.9140893470790378
Sensitivity (Recall): 0.8896321070234113
ROC AUC: 0.9679144634816241


In [45]:
import shap
import numpy as np
import pandas as pd

# Define feature names properly
if isinstance(X_train, pd.DataFrame):
    feature_names = X_train.columns.tolist()  # Extract feature names if X_train is a DataFrame
elif isinstance(X_train, np.ndarray):
    # Manually define feature names if using a NumPy array
    feature_names = [f'Feature_{i}' for i in range(X_train.shape[1])]

# Ensure X_train is a DataFrame with proper column names
if isinstance(X_train, np.ndarray):
    # Adjust feature_names if needed
    if len(feature_names) != X_train.shape[1]:
        feature_names = feature_names[:X_train.shape[1]]  # Adjust if too many
        feature_names.extend([f'Feature_{i}' for i in range(len(feature_names), X_train.shape[1])])  # Add if too few

    X_train = pd.DataFrame(X_train, columns=feature_names)

# Create a SHAP explainer
explainer = shap.Explainer(xgb_classifier)
shap_values = explainer(X_train)

# Get the SHAP values in a DataFrame with actual column names
shap_df = pd.DataFrame(shap_values.values, columns=feature_names)

# Calculate the mean absolute SHAP values for each feature
mean_shap_values = np.abs(shap_df).mean()

# Sort the features based on the mean absolute SHAP values
top_features = mean_shap_values.sort_values(ascending=False).head(6)

# Print the average SHAP values for the top 6 features
print("Average SHAP values for the top 6 features:")
print(top_features)

Average SHAP values for the top 6 features:
LogP                  1.094681
W                     0.450942
hydrophobic_moment    0.418986
E                     0.301181
R                     0.245926
N                     0.216463
dtype: float32
